# Análisis de Red de Tiendas - RetailNow

Este notebook analiza los datos de ventas, inventarios y satisfacción del cliente para la cadena de tiendas minoristas RetailNow.

## 1. Preparar el entorno de trabajo

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

## 2. Cargar los datos

In [ ]:
# Rutas de los archivos CSV - Rutas absolutas del proyecto
ruta_sales = '/workspace/sales.csv'
ruta_inventories = '/workspace/inventories.csv'
ruta_satisfaction = '/workspace/satisfaction.csv'

# Cargar los datos en DataFrames
df_sales = pd.read_csv(ruta_sales)
df_inventories = pd.read_csv(ruta_inventories)
df_satisfaction = pd.read_csv(ruta_satisfaction)

# Validar estructura de los DataFrames cargados
assert len(df_sales.columns) >= 4, "El CSV de ventas debe tener al menos 4 columnas"
assert len(df_inventories.columns) >= 3, "El CSV de inventarios debe tener al menos 3 columnas"
assert len(df_satisfaction.columns) >= 2, "El CSV de satisfacción debe tener al menos 2 columnas"

print("Datos cargados correctamente")
print(f"Ventas: {len(df_sales)} registros")
print(f"Inventarios: {len(df_inventories)} registros")
print(f"Satisfacción: {len(df_satisfaction)} registros")

## 3. Limpieza de datos

In [ ]:
# Mostrar valores nulos antes de limpiar
print("Valores nulos por DataFrame (antes de limpiar):")
print(f"  Ventas: {df_sales.isnull().sum().sum()}")
print(f"  Inventarios: {df_inventories.isnull().sum().sum()}")
print(f"  Satisfacción: {df_satisfaction.isnull().sum().sum()}")

# Eliminar filas con valores nulos
df_sales = df_sales.dropna()
df_inventories = df_inventories.dropna()
df_satisfaction = df_satisfaction.dropna()

print("\nDatos después de eliminar valores nulos:")
print(f"Ventas: {len(df_sales)} registros")
print(f"Inventarios: {len(df_inventories)} registros")
print(f"Satisfacción: {len(df_satisfaction)} registros")

## 4. Exploración de datos - Ventas

In [ ]:
# Calcular ventas totales por producto
ventas_por_producto = df_sales.groupby('Producto')['Cantidad_Vendida'].sum()
print("Ventas totales por producto:")
print(ventas_por_producto)
print()

In [ ]:
# Calcular ingresos totales por tienda
df_sales['Total_Venta'] = df_sales['Cantidad_Vendida'] * df_sales['Precio_Unitario']
ingresos_por_tienda = df_sales.groupby('ID_Tienda')['Total_Venta'].sum()
print("Ingresos totales por tienda:")
print(ingresos_por_tienda)
print()

In [ ]:
# Resumen estadístico de las ventas
resumen_ventas = df_sales['Total_Venta'].describe()
print("Resumen estadístico de ventas:")
print(resumen_ventas)
print()

In [ ]:
# Ventas totales por tienda
ventas_por_tienda = df_sales.groupby('ID_Tienda')['Cantidad_Vendida'].sum()
print("Ventas totales por tienda:")
print(ventas_por_tienda)
print()

## 5. Análisis de Inventarios

In [ ]:
# Calcular la rotación de inventarios por tienda
# Rotación = Ventas / Stock disponible

# Primero sumamos el stock disponible por tienda
stock_por_tienda = df_inventories.groupby('ID_Tienda')['Stock_Disponible'].sum()
print("Stock disponible por tienda:")
print(stock_por_tienda)
print()

In [ ]:
# Calcular porcentaje de ventas respecto al inventario
# Unimos los datos de ventas y stock por tienda
ventas_tienda = df_sales.groupby('ID_Tienda')['Cantidad_Vendida'].sum()
stock_tienda = df_inventories.groupby('ID_Tienda')['Stock_Disponible'].sum()

# Crear DataFrame con los datos combinados
df_rotacion = pd.DataFrame({
    'Ventas': ventas_tienda,
    'Stock': stock_tienda
})

# Calcular porcentaje de rotación (ventas / stock * 100)
df_rotacion['Porcentaje_Rotacion'] = (df_rotacion['Ventas'] / df_rotacion['Stock']) * 100

print("Rotación de inventarios por tienda:")
print(df_rotacion)
print()

In [ ]:
# Filtrar tiendas con inventarios críticos (menos del 10% de ventas respecto al inventario)
tiendas_criticas = df_rotacion[df_rotacion['Porcentaje_Rotacion'] < 10]
print("Tiendas con niveles críticos de inventario (< 10%):")
if len(tiendas_criticas) > 0:
    print(tiendas_criticas)
else:
    print("No hay tiendas con inventario crítico")
print()

## 6. Satisfacción del Cliente

In [ ]:
# Análisis de satisfacción del cliente por tienda
print("Satisfacción del cliente por tienda:")
print(df_satisfaction)
print()

In [ ]:
# Filtrar tiendas con baja satisfacción (< 60%)
tiendas_baja_satisfaccion = df_satisfaction[df_satisfaction['Satisfacción_Promedio'] < 60]
print("Tiendas con satisfacción menor al 60%:")
if len(tiendas_baja_satisfaccion) > 0:
    print(tiendas_baja_satisfaccion)
    print()
    print("Recomendaciones para mejorar:")
    for _, row in tiendas_baja_satisfaccion.iterrows():
        print(f"  - Tienda {row['ID_Tienda']}: Mejorar atención al cliente y calidad del servicio")
else:
    print("No hay tiendas con satisfacción baja")
print()

## 7. Operaciones con NumPy

In [ ]:
# Convertir columna Total_Ventas a array de NumPy
ventas_array = df_sales['Total_Venta'].to_numpy()

# Calcular la mediana de las ventas totales usando NumPy
mediana_ventas = np.median(ventas_array)
print(f"Mediana de las ventas totales: ${mediana_ventas:.2f}")
print()

In [ ]:
# Calcular la desviación estándar de las ventas usando NumPy
desviacion_estandar = np.std(ventas_array)
print(f"Desviación estándar de las ventas: ${desviacion_estandar:.2f}")
print()

## 8. Simulación de Proyecciones de Ventas Futuras

In [ ]:
# Simular proyecciones de ventas futuras usando NumPy
# Usamos distribución normal porque es un modelo estándar para simular variaciones
# naturales alrededor de un promedio, apropiado para proyectar ventas que fluctúan
# alrededor de un valor histórico medio
np.random.seed(42)

# Calcular el promedio de ventas actuales
ventas_promedio = np.mean(ventas_array)
ventas_std = np.std(ventas_array)

# Generar 12 meses de proyecciones de ventas
num_proyecciones = 12
proyecciones = np.random.normal(ventas_promedio, ventas_std, num_proyecciones)

print("Proyecciones de ventas para los próximos 12 meses:")
for i, venta in enumerate(proyecciones, 1):
    print(f"  Mes {i}: ${max(0, venta):.2f}")  # No puede haber ventas negativas
print()

In [ ]:
# Calcular estadísticas de las proyecciones
proyecciones_positivas = np.maximum(proyecciones, 0)  # Asegurar valores positivos

print("Estadísticas de las proyecciones:")
print(f"  Promedio proyectado: ${np.mean(proyecciones_positivas):.2f}")
print(f"  Mediana proyectada: ${np.median(proyecciones_positivas):.2f}")
print(f"  Desviación estándar: ${np.std(proyecciones_positivas):.2f}")
print(f"  Ventas mínimas proyectadas: ${np.min(proyecciones_positivas):.2f}")
print(f"  Ventas máximas proyectadas: ${np.max(proyecciones_positivas):.2f}")

## Conclusiones

Este análisis proporciona una visión completa del rendimiento de las tiendas RetailNow, incluyendo:
- Ventas totales e ingresos por tienda
- Rotación de inventarios e identificación de tiendas con inventario crítico
- Análisis de satisfacción del cliente
- Estadísticas avanzadas usando NumPy
- Proyecciones de ventas futuras